In [1]:
#(3090) Para 128 tokens(Bert) e 32000 vocab: 1000 = 6 seg, 10000 = 1min, 100000 = 10min, 1m = 100min, 10m = 16h, 100m = 160h ou 6.6 dias
#(3090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Bert): 1000 = 4.3s, 10000 = 43s, 100000 = 7.1m, 1m = 71min, 10m = 11.8h, 100m = 118h ou 5 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Electra): 1000 = 3.5s, 10000 = 35s, 100000 = 5.9m, 1m = 59min, 10m = 9.8h, 100m = 98h ou 4 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias

#HIPOTÉTICO:
#(4090) Para 24 tokens(tokenmonster salva 35%)e 24000 vocab(Electra): 1000 = , 10000 = , 100000 = 3.19m, 1m = 32min, 10m = 6.5h, 100m = 65h ou 2.7 dias

#losses:
#1m dataset, 24000 vocab electra, tokenmonster: iteration:  7800 , total_loss:  26.277032788594564
#100k dataset, 16000 vocab electra, tokenmonster: iteration:  4980 , total_loss:  24.833836555480957
#10m dataset, 16vocab electra , tokenmonster, 10 dom, iteration:  99990 , total_loss:  20.500934664408366


#1m dataset, dom_electra_discriminator_16kvocab_1m  = 40min . Loss: 24.15

In [1]:
%reload_ext autoreload
%autoreload 2

import torch
print(torch.__version__)

import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

model_path='contrastive_model/'
if not os.path.exists('models/'+model_path):
    os.mkdir('models/'+model_path)
# !pip install datasets transformers tqdm magic_timer pandas tokenizers matplotlib pynvml
torch.cuda.is_available()

2.0.1+cu117


True

## Setup

In [3]:
%env WANDB_START_METHOD=thread
%env WANDB_PROJECT=pretraining_BERT_the_notebook
import json
from pathlib import Path
from typing import Iterator
import time

import datasets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pynvml
import torch
from magic_timer import MagicTimer
from tokenizers import BertWordPieceTokenizer, Regex, normalizers
from tqdm import tqdm
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizerFast,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

# Print hardware information
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
gpu_name = pynvml.nvmlDeviceGetName(handle)
gpu_mem = pynvml.nvmlDeviceGetMemoryInfo(handle).total / (1024**2)
print(f"GPU: {gpu_name}, {gpu_mem} MiB")
print(f"{torch.cuda.is_available() = }")
model_training='electra'#'bert' , 'electra',; modular
# LIMIT_DATASET = 2016 * 4  # keep small for development, set to None for full dataset


LIMIT_DATASET = 10_000
RANDOM_SEED = 42
NUM_TOKENIZER_TRAINING_ITEMS = 1_000_000  # I made this up, but it seems reasonable
if model_training=='bert':
    VOCAB_SIZE = 32_768  # from Cramming
    DEVICE_BATCH_SIZE = 100 # aprox 128, adjust to get near 100% gpu memory use
    MODEL_MAX_SEQ_LEN = 128  # from Cramming
else:
    VOCAB_SIZE = 16_000  # tokenmonster
    DEVICE_BATCH_SIZE = 128 # Token monster aguenta um batch size de (200-248)!! Geralmente melhora a qualidade do treino
    MODEL_MAX_SEQ_LEN = 84  # token_monster

N_DOMAINS=0
HIER_LEVEL=0

MASK_ID=4
PAD_ID=0

gradient_accumulation_steps = 2048 // DEVICE_BATCH_SIZE  # roughly based on Cramming
batch_size = DEVICE_BATCH_SIZE * gradient_accumulation_steps
print(f"{DEVICE_BATCH_SIZE = }")
print(f"{gradient_accumulation_steps = }")
print(f"{batch_size = }")
RUN_DIR = Path("data") / f"run_{time.strftime('%Y%m%d-%H%M%S')}"
CHECKPOINT_DIR = RUN_DIR / "training_checkpoints"
MODEL_DIR = RUN_DIR / "model"
TOKENIZER_PATH = RUN_DIR / "tokenizer.json"
TRAINER_HISTORY_PATH = RUN_DIR / "trainer_history.json"

RUN_DIR.mkdir(exist_ok=True, parents=True)

with MagicTimer() as timer:
    dataset = datasets.load_dataset(
        "sradc/chunked-shuffled-wikipedia20220301en-bookcorpusopen",
        split=f"train[:{LIMIT_DATASET}]" if LIMIT_DATASET else "train",
        revision="0e6fada2dd43136e4a3f637da41de2e596aee674",
    )
print(f"Loaded dataset in {timer}")
len(dataset)

env: WANDB_START_METHOD=thread
env: WANDB_PROJECT=pretraining_BERT_the_notebook
GPU: NVIDIA GeForce RTX 3090, 24576.0 MiB
torch.cuda.is_available() = True
DEVICE_BATCH_SIZE = 128
gradient_accumulation_steps = 16
batch_size = 2048


Found cached dataset parquet (/home/kiki/.cache/huggingface/datasets/sradc___parquet/sradc--chunked-shuffled-wikipedia20220301en-bookcorpusopen-ff5cb88917a65ec5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Loaded dataset in 1.7 seconds


10000

## Tokenize

In [4]:
# !pip install tokenmonster
from random import shuffle

tokenizer = BertWordPieceTokenizer()
tokenizer._tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace(Regex("(``|'')"), '"'),
        normalizers.NFD(),
        normalizers.Lowercase(),
        normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "),
        normalizers.Replace(Regex(r"[^\x00-\x7F]+"), ""),
    ]
)  # Normalizer based on, https://github.com/JonasGeiping/cramming/blob/50bd06a65a4cd4a3dd6ee9ecce1809e1a9085374/cramming/data/tokenizer_preparation.py#L52
def tokenizer_training_data() -> Iterator[str]:
    for i in tqdm(
        range(min(NUM_TOKENIZER_TRAINING_ITEMS, len(dataset))),
        desc="Feeding samples to tokenizer",
    ):
        yield dataset[i]["text"]


with MagicTimer() as timer:
    tokenizer.train_from_iterator(
        tokenizer_training_data(),
        vocab_size=VOCAB_SIZE,
        min_frequency=2,
    )
print(f"Tokenizer trained in {timer}.")
tokenizer.save(str(TOKENIZER_PATH))
tokenizer = BertTokenizerFast(tokenizer_file=str(TOKENIZER_PATH))
# tokenizer.unk_token,tokenizer.unk_token_id,tokenizer.sep_token,tokenizer.sep_token_id,tokenizer.pad_token,tokenizer.pad_token_id,tokenizer.cls_token,tokenizer.cls_token_id,tokenizer.mask_token,tokenizer.mask_token_id

class HFTokenizedDataset(torch.utils.data.Dataset):
    "This wraps the dataset and tokenizes it, ready for the model"

    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        inps= self.tokenizer.encode(
            self.dataset[i]["text"],
            return_tensors="pt",
            truncation=True,
            max_length=MODEL_MAX_SEQ_LEN - 2,
            padding="max_length",
            return_special_tokens_mask=True,
        )[0, ...]
        return {'input_ids':inps}

class TokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)
    
    def convert_tokens_to_ids(self,s):
        s=norm.normalize_str(s)
        tokens = vocab.tokenize(s)
        tokens=tokens[:MODEL_MAX_SEQ_LEN - 2]
        tokens=torch.Tensor(tokens).long()
        return tokens

    def __getitem__(self, i):
        s=norm.normalize_str(self.dataset[i]["text"])
        tokens = vocab.tokenize(s).tolist()
        
        #trucate
        tokens=tokens[:MODEL_MAX_SEQ_LEN - 2]
        l=len(tokens)
        for j in range(l,MODEL_MAX_SEQ_LEN - 2):
            tokens.append(0)

        tokens=torch.Tensor(tokens)
        if MODEL_MAX_SEQ_LEN - 2>l:
            att_mask=np.concatenate((np.ones(l),np.zeros(MODEL_MAX_SEQ_LEN - 2-l)))
        else:
            att_mask=np.ones(tokens.shape[0])
        # print(l,att_mask.shape[0],MODEL_MAX_SEQ_LEN - 2)
        assert tokens.shape[0]==att_mask.shape[0]

        input_ids=tokens

        input_ids=torch.as_tensor(input_ids,dtype=torch.long)
        # labels = torch.zeros(input_ids.shape)
        # probability_matrix = torch.full(labels.shape, 0.15)
        # masked_indices = torch.bernoulli(probability_matrix).bool()
        # indices_replaced = torch.bernoulli(torch.full(labels.shape, 1.0)).bool() & masked_indices
        # labels[indices_replaced] =1
        # mask=torch.as_tensor(labels,dtype=torch.bool)
        
        d={'input_ids':input_ids, 
            # 'token_type_ids':token_type_ids, 
            # 'attention_mask':attention_mask,
            # 'mask':mask
            'domain':0,
            'subdomain1':0,
            }
        return d

def texts2mlm(texts,domain,subdomain1=0):
    input_ids=[]
    # token_type_ids=[]
    # attention_mask=[]
    # data=[]
    inputs=[]
    # masks=[]
    for t in texts:
        s=norm.normalize_str(t["text"])
        tokens = vocab.tokenize(s).tolist()
        
        #trucate
        tokens=tokens[:MODEL_MAX_SEQ_LEN - 2]
        l=len(tokens)
        for j in range(l,MODEL_MAX_SEQ_LEN - 2):
            tokens.append(0)

        tokens=torch.Tensor(tokens)
        if MODEL_MAX_SEQ_LEN - 2>l:
            att_mask=np.concatenate((np.ones(l),np.zeros(MODEL_MAX_SEQ_LEN - 2-l)))
        else:
            att_mask=np.ones(tokens.shape[0])
        # print(l,att_mask.shape[0],MODEL_MAX_SEQ_LEN - 2)
        assert tokens.shape[0]==att_mask.shape[0]

        input_ids=tokens

        input_ids=torch.as_tensor(input_ids,dtype=torch.long)
        # labels = torch.zeros(input_ids.shape)
        # probability_matrix = torch.full(labels.shape, 0.15)
        # masked_indices = torch.bernoulli(probability_matrix).bool()
        # indices_replaced = torch.bernoulli(torch.full(labels.shape, 1.0)).bool() & masked_indices
        # labels[indices_replaced] =1
        # mask=torch.as_tensor(labels,dtype=torch.bool)
        inputs.append(input_ids)
        # masks.append(mask)
    
    return {'input_ids':torch.stack(inputs),'domain':domain,'subdomain1':(domain*N_DOMAINS)+subdomain1}

def divide_chunks(l, n):
    return [l[i:i + n] for i in range(0, len(l), n)] 

class CustomDataloader(torch.utils.data.Dataset):
    def __init__(self,dataset,cluster_labels,n_domains) -> None:
        self.dataset=dataset
        self.cluster_labels=cluster_labels
        self.bin_dataset={}
        self.domains=[i for i in range(n_domains)]
        self.batch_ordering=[]
        self.current_domain=0
        self.bs=DEVICE_BATCH_SIZE
        self.fill_bins()

    def fill_bins(self):
        self.bin_dataset={}
        for i,c in enumerate(self.cluster_labels):
            if c not in self.bin_dataset:
                self.bin_dataset[c]=[i]
            else:
                self.bin_dataset[c].append(i)
        n_batches=0
        domains=[]
        for i in range(len(self.domains)):
            self.bin_dataset[i]=divide_chunks(self.bin_dataset[i],DEVICE_BATCH_SIZE)
            for k in range(len(self.bin_dataset[i])):
                domains.append((i,k))
        shuffle(domains)
        self.batch_ordering=domains

    def __getitem__(self, i) -> torch.Tensor:
        indexes=self.bin_dataset[self.batch_ordering[i][0]][self.batch_ordering[i][1]]
        batch_data=[]
        for j in indexes:
            batch_data.append(self.dataset[j])
        # print('batch_data:',batch_data)
        batch_data=texts2mlm(batch_data,self.batch_ordering[i][0])
        # batch_data = torch.from_numpy(a).long()
        return batch_data

    def __len__(self):
        #colocar uma margem de erro pra baixo
        return len(self.batch_ordering)


class CustomDataloaderLevel1(torch.utils.data.Dataset):
    def __init__(self,dataset,cluster_labels,idxs_cluster_labels_level1,n_domains,n_subdomains) -> None:
        self.dataset=dataset
        self.cluster_labels=cluster_labels
        self.idxs_cluster_labels_level1=idxs_cluster_labels_level1
        self.bin_dataset={}
        self.domains=[i for i in range(n_domains)]
        self.subdomains=[i for i in range(n_subdomains)]
        self.batch_ordering=[]
        self.current_domain=0
        self.bs=DEVICE_BATCH_SIZE
        self.fill_bins()

    def fill_bins(self):
        self.bin_dataset={}
        # for i,c in enumerate(self.cluster_labels):
        #     if c not in self.bin_dataset:
        #         self.bin_dataset[c]=[i]
        #     else:
        #         self.bin_dataset[c].append(i)
        for dom,txt_id,subdom in self.idxs_cluster_labels_level1:
            id_dom_sub=str(dom)+'_'+str(subdom)#identifica o dominio e o subdominio
            if  id_dom_sub not in self.bin_dataset:
                self.bin_dataset[id_dom_sub]=[txt_id]
            else:
                self.bin_dataset[id_dom_sub].append(txt_id)

        n_batches=0
        data_hier_idxs=[]
        for i in range(len(self.domains)):
            for j in range(len(self.subdomains)):
                id_dom_sub=str(i)+'_'+str(j)#identifica o dominio e o subdominio
                #OBS: subdivide a lista de todos os ids do msm dom e subdom em batches. O bin dataset e um dicionario
                #que mapeia dom e subdom -> lista de batches
                self.bin_dataset[id_dom_sub]=divide_chunks(self.bin_dataset[id_dom_sub],DEVICE_BATCH_SIZE)
                for k in range(len(self.bin_dataset[id_dom_sub])):
                    data_hier_idxs.append((id_dom_sub,k))
        shuffle(data_hier_idxs)
        #batch ordering é um vetor de tuplas de 2 elementos. O primeiro é o dominio e subdominio desse batch
        # O segundo elemento é a posicao do batch no bin dataset. 
        # Ex: ('0_0',0) se lê: Batch com dominio e subdominio 0, esse batch é o numero 0 na lista de batches do bin_dataset para esse dom e subdom.
        self.batch_ordering=data_hier_idxs

    def __getitem__(self, i) -> torch.Tensor:
        #o bin dataset tem como chaves o dominio e subdominio, e o valor é uma lista de batches de msm dom e subdom
        indexes=self.bin_dataset[self.batch_ordering[i][0]][self.batch_ordering[i][1]]
        batch_data=[]
        for j in indexes:
            batch_data.append(self.dataset[int(j)])
        # print('batch_data:',batch_data)
        if len(self.subdomains)>0:
            d0,d1=self.batch_ordering[i][0].split('_')
            batch_data=texts2mlm(batch_data,int(d0),int(d1))
        else:
            batch_data=texts2mlm(batch_data,self.batch_ordering[i][0])
        # batch_data = torch.from_numpy(a).long()
        return batch_data

    def __len__(self):
        #colocar uma margem de erro pra baixo
        return len(self.batch_ordering)

def get_vocab():
    #### TokenMonster BRRR!!!
    import tokenmonster
    vocab = tokenmonster.load("englishcode-16000-balanced-v1")
    # vocab = TokenMonster.load('tokenizers_monster/english-24000-capcode.vocab')

    norm=normalizers.Sequence(
        [
            normalizers.Replace(Regex("(``|'')"), '"'),
            normalizers.NFD(),
            normalizers.Lowercase(),
            normalizers.StripAccents(),
            normalizers.Replace(Regex(" {2,}"), " "),
            normalizers.Replace(Regex(r"[^\x00-\x7F]+"), ""),
        ]
    )
    vocab.modify("[EOS]")
    vocab.modify("[UNK]")
    vocab.modify("[SEP]")
    vocab.modify("[PAD]")
    vocab.modify("[CLS]")
    vocab.modify("[MASK]")
    return norm,vocab
norm,vocab=get_vocab()
MASK_ID=vocab.tokenize("[MASK]")[0]
PAD_ID=vocab.tokenize("[PAD]")[0]

if model_training=='bert':
    tokenized_dataset = HFTokenizedDataset(dataset, tokenizer)
elif model_training=='electra':
    tokenized_dataset = TokenizedDataset(dataset)

# tokenizer.mask_token,tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

Feeding samples to tokenizer:   0%|          | 0/1000000 [00:00<?, ?it/s]

Feeding samples to tokenizer: 100%|██████████| 1000000/1000000 [00:44<00:00, 22397.39it/s]





Tokenizer trained in 57 seconds.


## Electra

### Architecture

In [5]:
import math
from functools import reduce
from collections import namedtuple

import torch
from torch import nn
import torch.nn.functional as F

# constants

Results = namedtuple('Results', [
    'loss',
    'mlm_loss',
    'disc_loss',
    'gen_acc',
    'disc_acc',
    'disc_labels',
    'disc_predictions'
])

# helpers
def log(t, eps=1e-9):
    return torch.log(t + eps)

def gumbel_noise(t):
    noise = torch.zeros_like(t).uniform_(0, 1)
    return -log(-log(noise))

def gumbel_sample(t, temperature = 1.):
    return ((t / temperature) + gumbel_noise(t)).argmax(dim=-1)

def prob_mask_like(t, prob):
    return torch.zeros_like(t).float().uniform_(0, 1) < prob

def mask_with_tokens(t, token_ids):
    init_no_mask = torch.full_like(t, False, dtype=torch.bool)
    mask = reduce(lambda acc, el: acc | (t == el), token_ids, init_no_mask)
    return mask

def get_mask_subset_with_prob(mask, prob):
    batch, seq_len, device = *mask.shape, mask.device
    max_masked = math.ceil(prob * seq_len)

    num_tokens = mask.sum(dim=-1, keepdim=True)
    mask_excess = (mask.cumsum(dim=-1) > (num_tokens * prob).ceil())
    mask_excess = mask_excess[:, :max_masked]

    rand = torch.rand((batch, seq_len), device=device).masked_fill(~mask, -1e9)
    _, sampled_indices = rand.topk(max_masked, dim=-1)
    sampled_indices = (sampled_indices + 1).masked_fill_(mask_excess, 0)

    new_mask = torch.zeros((batch, seq_len + 1), device=device)
    new_mask.scatter_(-1, sampled_indices, 1)
    return new_mask[:, 1:].bool()


def device_as(t1, t2):
   """
   Moves t1 to the device of t2
   """
   return t1.to(t2.device)

class ContrastiveLoss(nn.Module):
    
   """
   Vanilla Contrastive loss, also called InfoNceLoss as in SimCLR paper
   """
   def __init__(self, batch_size, temperature=0.5):
       super().__init__()
       self.batch_size = batch_size
       self.temperature = temperature
       self.mask = (~torch.eye(batch_size * 2, batch_size * 2, dtype=bool)).float()

   def calc_similarity_batch(self, a, b):
       representations = torch.cat([a, b], dim=0)
       return F.cosine_similarity(representations.unsqueeze(1), representations.unsqueeze(0), dim=2)

   def forward(self, proj_1, proj_2):
       """
       proj_1 and proj_2 are batched embeddings [batch, embedding_dim]
       where corresponding indices are pairs
       z_i, z_j in the SimCLR paper
       """
       batch_size = proj_1.shape[0]
       z_i = F.normalize(proj_1, p=2, dim=1)
       z_j = F.normalize(proj_2, p=2, dim=1)

       similarity_matrix = self.calc_similarity_batch(z_i, z_j)

       sim_ij = torch.diag(similarity_matrix, batch_size)
       sim_ji = torch.diag(similarity_matrix, -batch_size)

       positives = torch.cat([sim_ij, sim_ji], dim=0)

       nominator = torch.exp(positives / self.temperature)

       denominator = device_as(self.mask, similarity_matrix) * torch.exp(similarity_matrix / self.temperature)

       all_losses = -torch.log(nominator / torch.sum(denominator, dim=1))
       loss = torch.sum(all_losses) / (2 * self.batch_size)
       return loss

class SimCLR_Loss(nn.Module):
    def __init__(self, batch_size, temperature):
        super().__init__()
        self.batch_size = batch_size
        self.temperature = temperature

        self.mask = self.mask_correlated_samples(batch_size)
        self.criterion = nn.CrossEntropyLoss(reduction="sum")
        self.similarity_f = nn.CosineSimilarity(dim=2)

    def mask_correlated_samples(self, batch_size):
        N = 2 * batch_size
        mask = torch.ones((N, N), dtype=bool)
        mask = mask.fill_diagonal_(0)
        
        for i in range(batch_size):
            mask[i, batch_size + i] = 0
            mask[batch_size + i, i] = 0
        return mask

    def forward(self, z_i, z_j):

        N = 2 * self.batch_size

        z = torch.cat((z_i, z_j), dim=0)

        sim = self.similarity_f(z.unsqueeze(1), z.unsqueeze(0)) / self.temperature

        sim_i_j = torch.diag(sim, self.batch_size)
        sim_j_i = torch.diag(sim, -self.batch_size)
        
        # We have 2N samples, but with Distributed training every GPU gets N examples too, resulting in: 2xNxN
        positive_samples = torch.cat((sim_i_j, sim_j_i), dim=0).reshape(N, 1)
        negative_samples = sim[self.mask].reshape(N, -1)
        
        #SIMCLR
        labels = torch.from_numpy(np.array([0]*N)).reshape(-1).to(positive_samples.device).long() #.float()        
        logits = torch.cat((positive_samples, negative_samples), dim=1)
        loss = self.criterion(logits, labels)
        loss /= N
        
        return loss

# hidden layer extractor class, for magically adding adapter to language model to be pretrained
class HiddenLayerExtractor(nn.Module):
    def __init__(self, net, layer = -2):
        super().__init__()
        self.net = net
        self.layer = layer

        self.hidden = None
        self.hook_registered = False

    def _find_layer(self):
        if type(self.layer) == str:
            modules = dict([*self.net.named_modules()])
            return modules.get(self.layer, None)
        elif type(self.layer) == int:
            children = [*self.net.children()]
            return children[self.layer]
        return None

    def _hook(self, _, __, output):
        self.hidden = output

    def _register_hook(self):
        layer = self._find_layer()
        assert layer is not None, f'hidden layer ({self.layer}) not found'
        handle = layer.register_forward_hook(self._hook)
        self.hook_registered = True

    def forward(self, x):
        if self.layer == -1:
            return self.net(x['input'],x['domain'])

        if not self.hook_registered:
            self._register_hook()

        _ = self.net(x['input'],x['domain'])
        hidden = self.hidden
        self.hidden = None
        assert hidden is not None, f'hidden layer {self.layer} never emitted an output'
        return hidden
    
class Electra(nn.Module):
    def __init__(
        self,
        generator,
        discriminator,
        *,
        num_tokens = None,
        discr_dim = -1,
        discr_layer = -1,
        mask_prob = 0.15,
        replace_prob = 0.85,
        random_token_prob = 0.,
        mask_token_id = 2,
        pad_token_id = 0,
        mask_ignore_token_ids = [],
        disc_weight = 50.,
        gen_weight = 1.,
        temperature = 1.,
        ):
        super().__init__()

        self.generator = generator
        self.discriminator = discriminator

        if discr_dim > 0:
            self.discriminator = nn.Sequential(
                HiddenLayerExtractor(discriminator, layer = discr_layer),
                nn.Linear(discr_dim, 1)
            )

        # mlm related probabilities
        self.mask_prob = mask_prob
        self.replace_prob = replace_prob

        self.num_tokens = num_tokens
        self.random_token_prob = random_token_prob

        # token ids
        self.pad_token_id = pad_token_id
        self.mask_token_id = mask_token_id
        self.mask_ignore_token_ids = set([*mask_ignore_token_ids, pad_token_id])

        # sampling temperature
        self.temperature = temperature

        # loss weights
        self.disc_weight = disc_weight
        self.gen_weight = gen_weight


    def forward(self, inputs, **kwargs):
        input=inputs['input']
        b, t = input.shape

        replace_prob = prob_mask_like(input, self.replace_prob)

        # do not mask [pad] tokens, or any other tokens in the tokens designated to be excluded ([cls], [sep])
        # also do not include these special tokens in the tokens chosen at random
        no_mask = mask_with_tokens(input, self.mask_ignore_token_ids)
        mask = get_mask_subset_with_prob(~no_mask, self.mask_prob)

        # get mask indices
        mask_indices = torch.nonzero(mask, as_tuple=True)

        # mask input with mask tokens with probability of `replace_prob` (keep tokens the same with probability 1 - replace_prob)
        masked_input = input.clone().detach()

        # set inverse of mask to padding tokens for labels
        gen_labels = input.masked_fill(~mask, self.pad_token_id)

        # clone the mask, for potential modification if random tokens are involved
        # not to be mistakened for the mask above, which is for all tokens, whether not replaced nor replaced with random tokens
        masking_mask = mask.clone()

        # if random token probability > 0 for mlm
        if self.random_token_prob > 0:
            assert self.num_tokens is not None, 'Number of tokens (num_tokens) must be passed to Electra for randomizing tokens during masked language modeling'

            random_token_prob = prob_mask_like(input, self.random_token_prob)
            random_tokens = torch.randint(0, self.num_tokens, input.shape, device=input.device)
            random_no_mask = mask_with_tokens(random_tokens, self.mask_ignore_token_ids)
            random_token_prob &= ~random_no_mask
            masked_input = torch.where(random_token_prob, random_tokens, masked_input)

            # remove random token prob mask from masking mask
            masking_mask = masking_mask & ~random_token_prob

        # [mask] input
        masked_input = masked_input.masked_fill(masking_mask * replace_prob, self.mask_token_id)

        # get generator output and get mlm loss
        logits = self.generator(masked_input, **kwargs)

        mlm_loss = F.cross_entropy(
            logits.transpose(1, 2),
            gen_labels,
            ignore_index = self.pad_token_id
        )

        # use mask from before to select logits that need sampling
        sample_logits = logits[mask_indices]

        # sample
        sampled = gumbel_sample(sample_logits, temperature = self.temperature)

        # scatter the sampled values back to the input
        disc_input = input.clone()
        disc_input[mask_indices] = sampled.detach()

        # generate discriminator labels, with replaced as True and original as False
        disc_labels = (input != disc_input).float().detach()

        # get discriminator predictions of replaced / original
        non_padded_indices = torch.nonzero(input != self.pad_token_id, as_tuple=True)

        # get discriminator output and binary cross entropy loss
        disc_logits = self.discriminator({'input':disc_input,'domain':inputs['domain']}, **kwargs)
        disc_logits = disc_logits.reshape_as(disc_labels)

        disc_loss = F.binary_cross_entropy_with_logits(
            disc_logits[non_padded_indices],
            disc_labels[non_padded_indices]
        )

        # gather metrics
        with torch.no_grad():
            gen_predictions = torch.argmax(logits, dim=-1)
            disc_predictions = torch.round((torch.sign(disc_logits) + 1.0) * 0.5)
            gen_acc = (gen_labels[mask] == gen_predictions[mask]).float().mean()
            disc_acc = 0.5 * (disc_labels[mask] == disc_predictions[mask]).float().mean() + 0.5 * (disc_labels[~mask] == disc_predictions[~mask]).float().mean()

        # return weighted sum of losses
        return Results(self.gen_weight * mlm_loss + self.disc_weight * disc_loss, mlm_loss, disc_loss, gen_acc, disc_acc, disc_labels, disc_predictions)
    

class ElectraConstrative(nn.Module):
    def __init__(
        self,
        generator,
        discriminator,
        *,
        num_tokens = None,
        discr_dim = -1,
        discr_layer = -1,
        mask_prob = 0.15,
        replace_prob = 0.85,
        random_token_prob = 0.,
        mask_token_id = 2,
        pad_token_id = 0,
        mask_ignore_token_ids = [],
        disc_weight = 50.,
        gen_weight = 1.,
        temperature = 1.,
        ):
        super().__init__()

        self.generator = generator
        self.discriminator = discriminator

        if discr_dim > 0:
            self.discriminator = nn.Sequential(
                HiddenLayerExtractor(discriminator, layer = discr_layer),
                nn.Linear(discr_dim, 1)
            )

        # mlm related probabilities
        self.mask_prob = mask_prob
        self.replace_prob = replace_prob

        self.num_tokens = num_tokens
        self.random_token_prob = random_token_prob

        # token ids
        self.pad_token_id = pad_token_id
        self.mask_token_id = mask_token_id
        self.mask_ignore_token_ids = set([*mask_ignore_token_ids, pad_token_id])

        # sampling temperature
        self.temperature = temperature

        # loss weights
        self.disc_weight = disc_weight
        self.gen_weight = gen_weight

        # self.const_loss = SimCLR_Loss(batch_size=DEVICE_BATCH_SIZE, temperature=0.1)
        self.const_loss = ContrastiveLoss(batch_size=DEVICE_BATCH_SIZE, temperature=0.1)


    def forward(self, inputs, **kwargs):
        input=inputs['input']
        b, t = input.shape

        replace_prob = prob_mask_like(input, self.replace_prob)

        # do not mask [pad] tokens, or any other tokens in the tokens designated to be excluded ([cls], [sep])
        # also do not include these special tokens in the tokens chosen at random
        no_mask = mask_with_tokens(input, self.mask_ignore_token_ids)
        mask = get_mask_subset_with_prob(~no_mask, self.mask_prob)

        # get mask indices
        mask_indices = torch.nonzero(mask, as_tuple=True)

        # mask input with mask tokens with probability of `replace_prob` (keep tokens the same with probability 1 - replace_prob)
        masked_input = input.clone().detach()

        # set inverse of mask to padding tokens for labels
        gen_labels = input.masked_fill(~mask, self.pad_token_id)

        # clone the mask, for potential modification if random tokens are involved
        # not to be mistakened for the mask above, which is for all tokens, whether not replaced nor replaced with random tokens
        masking_mask = mask.clone()

        # if random token probability > 0 for mlm
        if self.random_token_prob > 0:
            assert self.num_tokens is not None, 'Number of tokens (num_tokens) must be passed to Electra for randomizing tokens during masked language modeling'

            random_token_prob = prob_mask_like(input, self.random_token_prob)
            random_tokens = torch.randint(0, self.num_tokens, input.shape, device=input.device)
            random_no_mask = mask_with_tokens(random_tokens, self.mask_ignore_token_ids)
            random_token_prob &= ~random_no_mask
            masked_input = torch.where(random_token_prob, random_tokens, masked_input)

            # remove random token prob mask from masking mask
            masking_mask = masking_mask & ~random_token_prob

        # [mask] input
        masked_input = masked_input.masked_fill(masking_mask * replace_prob, self.mask_token_id)

        # get generator output and get mlm loss
        logits = self.generator(masked_input, **kwargs)

        mlm_loss = F.cross_entropy(
            logits.transpose(1, 2),
            gen_labels,
            ignore_index = self.pad_token_id
        )

        # use mask from before to select logits that need sampling
        sample_logits = logits[mask_indices]

        # sample
        sampled = gumbel_sample(sample_logits, temperature = self.temperature)

        # scatter the sampled values back to the input
        disc_input = input.clone()
        disc_input[mask_indices] = sampled.detach()

        # generate discriminator labels, with replaced as True and original as False
        disc_labels = (input != disc_input).float().detach()

        # get discriminator predictions of replaced / original
        non_padded_indices = torch.nonzero(input != self.pad_token_id, as_tuple=True)

        
        #Contrastive Learning

        #get generator input/output embeddings from discriminator 
        # embeddings_1 = self.discriminator[0]({'input':input,'domain':inputs['domain']}, **kwargs)
        # embeddings_2 = self.discriminator[0]({'input':disc_input,'domain':inputs['domain']}, **kwargs)

        #simcse: both embeddings are from the same sentence(output from generator), with dropout as noise
        embeddings_1 = self.discriminator[0]({'input':disc_input,'domain':inputs['domain']}, **kwargs)
        embeddings_2 = self.discriminator[0]({'input':disc_input,'domain':inputs['domain']}, **kwargs)

        #average embeddings
        sent_embeddings_1 = torch.mean(embeddings_1, dim=1)
        sent_embeddings_2 = torch.mean(embeddings_2, dim=1)
        
        constrative_loss = self.const_loss.forward(sent_embeddings_1,sent_embeddings_2)
        # print(sent_embeddings_1)
        # print(sent_embeddings_2)
        # print(constrative_loss)
        # a=2/0

        # get discriminator output and binary cross entropy loss
        disc_logits = self.discriminator[1](embeddings_2)
        disc_logits = disc_logits.reshape_as(disc_labels)

        disc_loss = F.binary_cross_entropy_with_logits(
            disc_logits[non_padded_indices],
            disc_labels[non_padded_indices]
        )

        disc_loss+=disc_loss+constrative_loss


        # gather metrics
        with torch.no_grad():
            gen_predictions = torch.argmax(logits, dim=-1)
            disc_predictions = torch.round((torch.sign(disc_logits) + 1.0) * 0.5)
            gen_acc = (gen_labels[mask] == gen_predictions[mask]).float().mean()
            disc_acc = 0.5 * (disc_labels[mask] == disc_predictions[mask]).float().mean() + 0.5 * (disc_labels[~mask] == disc_predictions[~mask]).float().mean()

        # return weighted sum of losses
        return Results(self.gen_weight * mlm_loss + self.disc_weight * disc_loss, mlm_loss, disc_loss, gen_acc, disc_acc, disc_labels, disc_predictions)


### Train

In [6]:
# pip install x_transformers reformer_pytorch accelerate einops

import torch
from torch import nn
from reformer_pytorch import ReformerLM
from x_transformers import TransformerWrapper, Decoder,Encoder

# (1) instantiate the generator and discriminator, making sure that the generator is roughly a quarter to a half of the size of the discriminator


generator = TransformerWrapper(
    num_tokens = VOCAB_SIZE+8,
    max_seq_len = MODEL_MAX_SEQ_LEN,
    # emb_dropout = 0.1,
    attn_layers = Decoder(
        dim = 256,
        depth = 12,
        heads = 4,
        attn_flash = True,
        domains=int(math.sqrt(N_DOMAINS)),
        subdomains1=N_DOMAINS,
        # rel_pos_bias = True 
    )
)

discriminator = TransformerWrapper(
    num_tokens = VOCAB_SIZE+8,
    max_seq_len = MODEL_MAX_SEQ_LEN,
    # emb_dropout = 0.1,
    attn_layers = Decoder(
        dim = 768,
        depth = 12,
        heads = 12,
        attn_flash = True,
        domains=0,
        layer_dropout=0.1,
        cross_attn_tokens_dropout=0.1,
        # rel_pos_bias = True 
    )
)

#works with reformer!!!
# generator.token_emb = discriminator.token_emb
# generator.pos_emb = discriminator.pos_emb

model = ElectraConstrative(
    generator,
    discriminator,
    temperature=0.05,
    discr_dim = 768,           # the embedding dimension of the discriminator
    # discr_dim = 1024,           # the embedding dimension of the discriminator
    # discr_layer = 'reformer',   # the layer name in the discriminator, whose output would be used for predicting token is still the same or replaced
    discr_layer = 'attn_layers',   # the layer name in the discriminator, whose output would be used for predicting token is still the same or replaced
    mask_token_id = MASK_ID,          # the token id reserved for masking
    pad_token_id = PAD_ID,           # the token id for padding
    mask_prob = 0.15,           # masking probability for masked language modeling
    mask_ignore_token_ids = []  # ids of tokens to ignore for mask modeling ex. (cls, sep)
)

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


In [7]:
# input_ids are the indices corresponding to each token in the sentence.
# attention_mask indicates whether a token should be attended to or not.
# token_type_ids identifies which sequence a token belongs to when there is more than one sequence
import numpy as np


from torch.utils.data import DataLoader
if model_training=='modular':
    if HIER_LEVEL==0:
        train_dataloader = CustomDataloader(dataset,cluster_labels,N_DOMAINS)   
    elif HIER_LEVEL==1:
        #no dataloader os as var fr subdom sao do msm tamanho do dom. Exemplo: 3 dom e 3 subdom
        #No modelo é diferente, temos uma primeira camada com 3 branches(dominio), e uma segunda camada com 9 branches(subdominios)
        train_dataloader = CustomDataloaderLevel1(dataset,cluster_labels,idxs_cluster_labels_level1,int(math.sqrt(N_DOMAINS)),int(math.sqrt(N_DOMAINS)) ) 
else:
    train_dataloader = DataLoader(
            tokenized_dataset, shuffle=True, batch_size=DEVICE_BATCH_SIZE
        )
# Optimizer
learning_rate=5e-5
weight_decay=0

gradient_accumulation_steps=1
max_train_steps=None
num_train_epochs=1
lr_scheduler_type='linear'
num_warmup_steps=0
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=learning_rate)

import math 
from transformers import (
    get_scheduler,
)
# Scheduler and math around the number of training steps.
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
if max_train_steps is None:
    max_train_steps = num_train_epochs * num_update_steps_per_epoch
    overrode_max_train_steps = True

lr_scheduler = get_scheduler(
    name=lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps * gradient_accumulation_steps,
    num_training_steps=max_train_steps * gradient_accumulation_steps,
)
from accelerate import Accelerator

accelerator = Accelerator()
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)

In [8]:
import torch.nn.functional as F
from einops import rearrange, pack, unpack

# Only show the progress bar once on each machine.
# progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
starting_epoch = 0

# update the progress_bar if load from checkpoint
total_loss=0
# progress_bar.update(completed_steps)
steps_log=30
count_amostra=0
num_train_epochs=1
for epoch in range(starting_epoch, num_train_epochs):
    if model_training=='modular':
        train_dataloader.fill_bins()
    model.train()
    for step, batch in enumerate(train_dataloader):
        # with accelerator.accumulate(model):
        #test if batch is in the right format
        if len(batch['input_ids'])!=DEVICE_BATCH_SIZE:
            break
         
        results=model({'input':batch['input_ids'].cuda(),'domain':batch['domain'],'subdomain1':batch['subdomain1']}) 
        # results=model({'input':batch['input_ids'].cuda()}) 

        # count_amostra+=int(len(batch['input_ids']))
        loss = results.loss
        total_loss += loss.detach().float().cpu().numpy().item()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()       
        
        if step%steps_log==0:
            print('iteration: ',step,', total_loss: ',total_loss/steps_log)
            # print('count_amostra:',count_amostra)
            total_loss=0
    # Checks if the accelerator has performed an optimization step behind the scenes
    if accelerator.sync_gradients:
        # progress_bar.update(1)
        completed_steps += 1

    if completed_steps >= max_train_steps:
        break

    model.eval()
    losses = []

iteration:  0 , total_loss:  6.992696634928385
iteration:  30 , total_loss:  285.9026407877604
iteration:  60 , total_loss:  197.3063757578532
iteration:  90 , total_loss:  124.7024553934733
iteration:  120 , total_loss:  87.41490745544434
iteration:  150 , total_loss:  85.36539370218912
iteration:  180 , total_loss:  112.19822909037272
iteration:  210 , total_loss:  101.4834103902181
iteration:  240 , total_loss:  115.21927909851074
iteration:  270 , total_loss:  91.0132874806722
iteration:  300 , total_loss:  77.12401606241862
iteration:  330 , total_loss:  76.62908732096354
iteration:  360 , total_loss:  106.94939333597819
iteration:  390 , total_loss:  76.78578592936198
iteration:  420 , total_loss:  72.05549405415853
iteration:  450 , total_loss:  55.4007319132487
iteration:  480 , total_loss:  66.08126742045084
iteration:  510 , total_loss:  61.2308967590332
iteration:  540 , total_loss:  66.16464856465657
iteration:  570 , total_loss:  61.01326192220052
iteration:  600 , total_l

In [9]:
torch.save(model.discriminator,'models/contrastive_model/discriminator_16kvocab_1m.pt')

# Finetuning model

### Electra custom

In [10]:
import transformers
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import os
from torch.utils.data import  DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

    
class ElectraClass(torch.nn.Module):
    def __init__(self):
        super(ElectraClass, self).__init__()
        self.l1 = model.discriminator._modules['0']
        self.l2 = torch.nn.Dropout(0.3)
        self.fl = torch.nn.Flatten()
        self.l3 = torch.nn.Linear(768*(MODEL_MAX_SEQ_LEN-2), 3)
    
    def forward(self, ids):
        output= self.l1(ids)
        output = self.l2(output)
        output = self.fl(output)
        output = self.l3(output)
        return output

electra_model = ElectraClass()
electra_model=electra_model.to('cuda')
optimizer = torch.optim.Adam(params =  model.parameters(), lr=2e-5,weight_decay=0.01)
import numpy as np
from torch.autograd import Variable

electra_model.train()

def loss_fn(outputs, targets):
    loss=torch.nn.CrossEntropyLoss()(outputs, targets)
    return loss

In [11]:
task='mrpc'
dataset = load_dataset('glue', task, split='train')

In [12]:
import numpy as np
class TokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset,is_dom=False):
        self.dataset = dataset
        self.ls=[]
        self.is_dom=is_dom

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        s=norm.normalize_str(self.dataset["sentence1"][i])+'[SEP]'+norm.normalize_str(self.dataset["sentence2"][i])
        tokens = vocab.tokenize(s).tolist()        
        
        self.ls.append(i)
        
        #trucate
        tokens=tokens[:MODEL_MAX_SEQ_LEN - 2]
        l=len(tokens)
        for j in range(l,MODEL_MAX_SEQ_LEN - 2):
            tokens.append(0)
        tokens=torch.Tensor(tokens).long()

        label=self.dataset["label"][i]

        if self.is_dom:
            p=tfidf_model.transform([self.dataset["sentence1"][i]+self.dataset["sentence2"][i]])
            dom = kmeans.predict(
                X=torch.from_numpy(p).to(device)
            )
            d={'input_ids':tokens,'label':label, 'domain':dom.item()}
        else:
            d={'input_ids':tokens,'label':label,'domain':0}
        return d

class HFTokenizedDataset(torch.utils.data.Dataset):
    "This wraps the dataset and tokenizes it, ready for the model"

    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        label=self.dataset["label"][i]
        inps=self.tokenizer.encode(
            self.dataset["sentence1"][i]+'[SEP]'+self.dataset["sentence2"][i],
            return_tensors="pt",
            truncation=True,
            max_length=MODEL_MAX_SEQ_LEN - 2,
            padding="max_length",
            return_special_tokens_mask=True,
        )[0, ...]
        return {'input_ids':inps,
            'label':label}

from torch.utils.data import DataLoader

if model_training=='bert':
    tokenized_dataset = HFTokenizedDataset(dataset, tokenizer)
else:
    tokenized_dataset = TokenizedDataset(dataset)
# train_dataloader=CustomDataloader(tokenized_dataset)
train_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
        # tokenized_dataset, batch_size=1
    )

In [13]:
epochs=2
running_loss=0
for i in range(epochs):
    for j,data in enumerate(train_dataloader):
        if model_training=='bert':
            ids = data['input_ids'].to('cuda', dtype = torch.long)
        else:
            ids = {'input':data['input_ids'].to('cuda', dtype = torch.long),'domain':0}
            # ids = {'input':data['input_ids'].to('cuda', dtype = torch.long),'domain':data['domain']}
            
        
        targets = data['label'].to('cuda', dtype = torch.long)

        outputs = electra_model(ids)
        
        loss = loss_fn(outputs,targets)
        # loss=outputs.loss

        optimizer.zero_grad()
        running_loss+=loss.item()
        if j%50==0:
            print(f'Epoch: {i}, Iteration: {j}, Loss:  {running_loss/50}')
            running_loss=0
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 0, Iteration: 0, Loss:  0.02487355947494507
Epoch: 1, Iteration: 0, Loss:  0.41004410028457644


In [14]:
dataset = load_dataset('glue', task, split='validation')
if model_training=='bert':
    tokenized_dataset = HFTokenizedDataset(dataset, tokenizer)
else:
    # tokenized_dataset = TokenizedDataset(dataset)
    tokenized_dataset = TokenizedDataset(dataset)

eval_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
        # tokenized_dataset, batch_size=1
    )

In [15]:
model.eval()
predictions=[]
labels=[]
c=0
for _,data in enumerate(eval_dataloader):
    if model_training=='bert':
        ids = data['input_ids'].to('cuda', dtype = torch.long)
    else:
        ids = {'input':data['input_ids'].to('cuda', dtype = torch.long),'domain':0}
        # ids = {'input':data['input_ids'].to('cuda', dtype = torch.long),'domain':data['domain']}
        
    targets = data['label'].to('cuda', dtype = torch.long)
    
    outputs = electra_model(ids)
    
    outputs=torch.argmax(outputs, dim=1).cpu()
    targets=targets.cpu()
    
    outputs=[int(o.item()) for o in outputs]
    targets=[int(o.item()) for o in targets]

    predictions+=outputs
    labels+=targets

In [16]:
from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.50      0.08      0.13       129
           1       0.69      0.96      0.81       279

    accuracy                           0.68       408
   macro avg       0.60      0.52      0.47       408
weighted avg       0.63      0.68      0.59       408



              precision    recall  f1-score   support

           0       0.44      0.25      0.32       129
           1       0.71      0.86      0.78       279

    accuracy                           0.66       408
   macro avg       0.58      0.55      0.55       408
weighted avg       0.63      0.66      0.63       408



In [ ]:
## glue mrpc com 3_dom_electra_discriminator_16kvocab_10m
                precision    recall  f1-score   support

           0       0.61      0.57      0.59       129
           1       0.81      0.83      0.82       279

    accuracy                           0.75       408
   macro avg       0.71      0.70      0.71       408
weighted avg       0.75      0.75      0.75       408

## glue mrpc com 3_dom_electra_discriminator_16kvocab_10m
         precision    recall  f1-score   support

           0       0.56      0.54      0.55       129
           1       0.79      0.81      0.80       279

    accuracy                           0.72       408
   macro avg       0.68      0.67      0.68       408
weighted avg       0.72      0.72      0.72       408

## glue mrpc com electra_discriminator_16kvocab_10m
              precision    recall  f1-score   support

           0       0.49      0.47      0.48       129
           1       0.76      0.78      0.77       279

    accuracy                           0.68       408
   macro avg       0.63      0.62      0.62       408
weighted avg       0.67      0.68      0.68       408

## glue mrpc com electra com pretraining de 1m:
            precision    recall  f1-score   support

           0       0.47      0.48      0.47       129
           1       0.76      0.75      0.75       279

    accuracy                           0.66       408
   macro avg       0.61      0.61      0.61       408
weighted avg       0.66      0.66      0.66       408


## glue mrpc com electra sem pretraining:
              precision    recall  f1-score   support

           0       0.50      0.23      0.32       129
           1       0.72      0.89      0.79       279

    accuracy                           0.68       408
   macro avg       0.61      0.56      0.56       408
weighted avg       0.65      0.68      0.64       408

### Bert custom

In [11]:
import transformers
import torch

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-cased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 3)
    
    def forward(self, ids, mask, token_type_ids):
        outs, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output
    

model = BERTClass()

In [3]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import transformers
import torch

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

In [ ]:
from torch.utils.data import  DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
task='mrpc'
dataset = load_dataset('glue', task, split='train')
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased', use_fast=True)
# dataset = dataset.map(lambda e: tokenizer(e['sentence1'], truncation=True, padding='max_length'), batched=True)
# dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label']) #BERT
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=12)

model=model.to('cuda')
optimizer = torch.optim.Adam(params =  model.parameters(), lr=2e-5,weight_decay=0.01)

Found cached dataset glue (/home/kiki/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True, padding='max_length')
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True, padding='max_length')
sentence1_key,sentence2_key=("sentence1", "sentence2")
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataloader = torch.utils.data.DataLoader(encoded_dataset, batch_size=12)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
from torch.autograd import Variable
def loss_fn(outputs, targets):
    # if task != "stsb":
    #     outputs = torch.argmax(outputs, dim=1)
    # else:
    #     outputs = outputs[:, 0]
    # loss=torch.nn.BCEWithLogitsLoss()(outputs,targets)
    # loss=torch.nn.CrossEntropyLoss()(outputs,targets)
    loss=torch.nn.CrossEntropyLoss()(outputs.view(-1,3), targets.view(-1))
    # loss = Variable(loss, requires_grad = True)
    return loss


In [ ]:
model.train()
epochs=5
running_loss=0
for i in range(epochs):
    for _,data in enumerate(dataloader):
        ids = data['input_ids'].to('cuda', dtype = torch.long)
        mask = data['attention_mask'].to('cuda', dtype = torch.long)
        token_type_ids = data['token_type_ids'].to('cuda', dtype = torch.long)
        targets = data['label'].to('cuda', dtype = torch.long)

        # outputs = model(ids, mask, token_type_ids)
        outputs = model(ids, mask, token_type_ids)
        loss = loss_fn(outputs.logits,targets)
        # loss=outputs.loss

        optimizer.zero_grad()
        running_loss+=loss.item()
        if _%50==0:
            print(f'Epoch: {i}, Loss:  {running_loss/200}')
            running_loss=0
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 0, Loss:  0.007852643132209777
Epoch: 0, Loss:  0.20191293969750404
Epoch: 0, Loss:  0.16229299038648606
Epoch: 0, Loss:  0.14223801746964454
Epoch: 0, Loss:  0.1337541800737381
Epoch: 0, Loss:  0.11794026836752891
Epoch: 0, Loss:  0.10376800172030926
Epoch: 1, Loss:  0.0131098260730505
Epoch: 1, Loss:  0.10432511921972036
Epoch: 1, Loss:  0.10928954795002938


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   │   │   running_loss=0                                                                  │
│   21 │   │                                                                                       │
│   22 │   │   optimizer.zero_grad()                                                               │
│ ❱ 23 │   │   loss.backward()                                                                     │
│   24 │   │   optimizer.step()                                                                    │
│   25                                                                                             │
│                                                                                                  │
│ /home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/torch/_tensor.py:487 in backward    │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_graph, inputs=inputs                     │
│    489 │   │   )                                                                                 │
│    490                                                                                           │
│                                                                                                  │
│ /home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/torch/autograd/__init__.py:200 in   │
│ backward                                                                                         │
│                                                                                                  │
│   197 │   # The reason we repeat same the comment below is that                                  │
│   198 │   # some Python versions print out the first line of a multi-line function               │
│   199 │   # calls in the traceback and some print out the last line                              │
│ ❱ 200 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   201 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                        │
│   202 │   │   allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to ru   │
│   203                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
dataset = load_dataset('glue', task, split='validation')
# dataset = dataset.map(lambda e: tokenizer(e['sentence1'], truncation=True, padding='max_length'), batched=True)
# dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=4)
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataloader = torch.utils.data.DataLoader(encoded_dataset, batch_size=12)

Found cached dataset glue (/home/kiki/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
model.eval()
predictions=[]
labels=[]
c=0
for _,data in enumerate(dataloader):
    ids = data['input_ids'].to('cuda', dtype = torch.long)
    mask = data['attention_mask'].to('cuda', dtype = torch.long)
    token_type_ids = data['token_type_ids'].to('cuda', dtype = torch.long)
    targets = data['label'].to('cuda', dtype = torch.long)

    outputs = model(ids, mask, token_type_ids)
    outputs = outputs.logits
    
    outputs=torch.argmax(outputs, dim=1).cpu()
    targets=targets.cpu()
    
    outputs=[int(o.item()) for o in outputs]
    targets=[int(o.item()) for o in targets]

    predictions+=outputs
    labels+=targets

In [ ]:
# from datasets import load_metric
# metric = load_metric('glue', 'mrpc')
# final_score = metric.compute(predictions=predictions, references =labels )

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.64      0.84      0.73       129
           1       0.92      0.78      0.84       279

    accuracy                           0.80       408
   macro avg       0.78      0.81      0.79       408
weighted avg       0.83      0.80      0.81       408



### With trainer

In [12]:
import transformers

print(transformers.__version__)

4.30.0.dev0


In [13]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [15]:
task = "mrpc"
model_checkpoint = "bert-base-cased"
batch_size = 16

In [16]:
from datasets import load_dataset, load_metric

In [17]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Found cached dataset glue (/home/kiki/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_244174/1389288479.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)


In [18]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [19]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
sentence1_key, sentence2_key = task_to_keys[task]


In [20]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [21]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [22]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

In [23]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [24]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [25]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/abcp4/bert-base-cased-finetuned-mrpc into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
trainer.train()

In [27]:
trainer.evaluate()

  0%|          | 0/26 [00:00<?, ?it/s]

{'eval_loss': 0.5459453463554382,
 'eval_accuracy': 0.8578431372549019,
 'eval_f1': 0.8993055555555555,
 'eval_runtime': 0.4866,
 'eval_samples_per_second': 838.425,
 'eval_steps_per_second': 53.429,
 'epoch': 5.0}

In [41]:
preds=trainer.predict(encoded_dataset["train"])

  0%|          | 0/230 [00:00<?, ?it/s]

In [28]:
dataset = load_dataset('glue', task, split='validation')
dataset = dataset.map(lambda e: tokenizer(e['sentence1'], truncation=True, padding='max_length'), batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4)

Found cached dataset glue (/home/kiki/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [29]:
model.eval()
predictions=[]
labels=[]
c=0
for _,data in enumerate(dataloader):
    ids = data['input_ids'].to('cuda', dtype = torch.long)
    mask = data['attention_mask'].to('cuda', dtype = torch.long)
    token_type_ids = data['token_type_ids'].to('cuda', dtype = torch.long)
    targets = data['label'].to('cuda', dtype = torch.long)

    outputs = model(ids, mask, token_type_ids)
    outputs = outputs.logits
    
    outputs=torch.argmax(outputs, dim=1).cpu()
    targets=targets.cpu()
    
    outputs=[int(o.item()) for o in outputs]
    targets=[int(o.item()) for o in targets]

    predictions+=outputs
    labels+=targets


In [47]:
predictions=np.argmax(preds.predictions, axis=1)
labels=encoded_dataset["train"]['label']
predictions

array([1, 1, 1, ..., 1, 1, 0])

In [48]:
# from datasets import load_metric
# metric = load_metric('glue', 'mrpc')
# final_score = metric.compute(predictions=predictions, references =labels )
from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1194
           1       0.99      1.00      0.99      2474

    accuracy                           0.99      3668
   macro avg       0.99      0.99      0.99      3668
weighted avg       0.99      0.99      0.99      3668



## Torch model to HuggingFaceTransformers(optional)

In [ ]:
# Torch model to HuggingFaceTransformers

In [ ]:
import torch.nn as nn
from transformers import PreTrainedModel, PretrainedConfig
from transformers import AutoModel, AutoConfig
from transformers import AutoModelForSequenceClassification
from typing import List, Optional, Tuple, Union
from transformers.modeling_outputs import (
    SequenceClassifierOutput,
)
import torch

In [ ]:
class MyConfig(PretrainedConfig):
    model_type = 'mymodel'
    def __init__(self, important_param=42, **kwargs):
        super().__init__(**kwargs)
        self.important_param = important_param

class MyModel(PreTrainedModel):
    config_class = MyConfig
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        self.model = nn.Sequential(
                          nn.Linear(3, self.config.important_param),
                          nn.Sigmoid(),
                          nn.Linear(self.config.important_param, 1),
                          nn.Sigmoid()
                          )
    def forward(self, input):
        return self.model(input) 

In [ ]:
config = MyConfig(4)
model = MyModel(config)
model.save_pretrained('./models/electra')

In [ ]:
AutoConfig.register("mymodel", MyConfig)
AutoModel.register(MyConfig, MyModel)

# new_model = MyModel.from_pretrained('./models/electra')
new_model = AutoModel.from_pretrained('./models/electra')
new_model

MyModel(
  (model): Sequential(
    (0): Linear(in_features=3, out_features=4, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=4, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

In [ ]:
# new_model.push_to_hub("mymodel-test")

In [ ]:
from torch.autograd import Variable

class MySeqConfig(PretrainedConfig):
    model_type = 'myseqmodel'
    def __init__(self, important_param=42, **kwargs):
        super().__init__(**kwargs)
        self.important_param = important_param

class MySequenceModel(PreTrainedModel):
    config_class = MySeqConfig
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        self.model = nn.Sequential(
                          nn.Linear(3, self.config.important_param),
                          nn.Sigmoid(),
                          nn.Linear(self.config.important_param, 1),
                          nn.Sigmoid()
                          )
    def forward(self, input_ids: Optional[torch.Tensor] = None,
                attention_mask: Optional[torch.Tensor] = None,
                token_type_ids: Optional[torch.Tensor] = None,
                position_ids: Optional[torch.Tensor] = None,
                head_mask: Optional[torch.Tensor] = None,
                inputs_embeds: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None,
                output_attentions: Optional[bool] = None,
                output_hidden_states: Optional[bool] = None,
                return_dict: Optional[bool] = None,)-> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        
        print(input_ids.shape)
        loss = Variable(torch.zeros(1).to('cuda'), requires_grad=True)

        
        return SequenceClassifierOutput(
            loss=loss,
            logits=torch.zeros((3)).to('cuda'),
            hidden_states=torch.zeros(128).to('cuda'),
            attentions=torch.zeros(128).to('cuda'),
        )


In [ ]:
config = MySeqConfig(4)
model = MySequenceModel(config)
model.save_pretrained('./models/electra')

In [ ]:
AutoModelForSequenceClassification.register(MySeqConfig, MySequenceModel)
AutoConfig.register("myseqmodel", MySeqConfig)

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained('./models/electra')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# Glue
!rm -r output_evals/runs
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
!python glue_eval.py --task_name cola --model_name_or_path bert-base-cased --tokenizer_name bert-base-cased --output_dir output_evals --do_train  --do_eval  --do_predict --max_seq_length 124 --per_device_train_batch_size 32 --learning_rate 2e-5 --lr_scheduler_type linear --num_train_epochs 5  --save_strategy no --seed 42 

06/24/2023 21:48:38 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
06/24/2023 21:48:38 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_ac